Construct initial cohort

In [ ]:
df_phe_condition_mapped = pd.read_csv('df_phe_condition_mapped.csv')

In [ ]:
# --- Define HS-related codes ---
HS_SNOMED_list = ['69741000', '59393003', '402826001', '402828000', '18638007']
HS_ICD_code = ['705.83', 'L73.2']
white_list = [
    "724834006", "1254916008", "275448003", "59393003", "18638007",
    "404176003", "402825002", "238992001", "403640008", "69741000",
    "402827005", "785724007", "238754005", "402826001", "1254915007",
    "402828000", "402706007", "404175004", "402752000", "74578003"
]

In [ ]:
# --- Identify HS case person_ids ---
df_hs_pid = df_phe_condition_mapped[
    df_phe_condition_mapped['standard_concept_code'].isin(HS_SNOMED_list) |
    df_phe_condition_mapped['ICDcode'].isin(HS_ICD_code)
][['person_id']].drop_duplicates()

# --- Identify control person_ids: not in HS group ---
df_control_condition = df_phe_condition_mapped[
    ~df_phe_condition_mapped['person_id'].isin(df_hs_pid['person_id'])
]

# --- Remove control individuals who appear in white-listed SNOMED codes ---
df_white_list_pid = df_control_condition[df_control_condition['standard_concept_code'].isin(white_list)]['person_id'].unique()
df_control_clean = df_control_condition[~df_control_condition['person_id'].isin(df_white_list_pid)]
df_control_clean_pid = df_control_clean[['person_id']].drop_duplicates()

# --- Final unique IDs ---
print('Number of case individuals:', len(df_hs_pid))
print('Number of control individuals:', df_control_clean['person_id'].nunique())

In [ ]:
# --- Select cohort
df_cohort = df_phe_condition_mapped[
    df_phe_condition_mapped['person_id'].isin(df_hs_pid['person_id']) |
    df_phe_condition_mapped['person_id'].isin(df_control_clean['person_id'])
]
df_cohort['HS'] = None
df_cohort.loc[df_cohort['person_id'].isin(df_hs_pid['person_id']), 'HS'] = 1
df_cohort.loc[df_cohort['person_id'].isin(df_control_clean['person_id']), 'HS'] = 0

In [ ]:
# --- Save PIDs
df_hs_pid.to_csv('HS_pid.csv', index=False)
df_control_clean_pid.to_csv('control_pid.csv', index=False)
df_cohort.to_csv('df_cohort.csv', index = False)